- P(w|wi-n+1) = Ct+lambda/N+B*lambda
on Ct es el numero d'aparicions del enegrama, N 

Només es cert en cas de que aquest caracter depengui dels n-1 anteriors. Quant mes gran sigui la sequencia, més real es aixó. El problema si el enegrama es massa gran el Ct serà cada cop més petit.  Si és més petit, és menys precís.

Fem servir smoothing pq hi ha enegramas que no estan al nostre corpus. 

- Posar doble espai al final i al inici del document per poder predir quin es el enegrama que indica l'inici/final (--a/a--)

- Altra tècnica de smoothing: Interpolació P(wi wi-1 wi-2) = lambda0 * P(wi, wi-1, wi-2) + lambda1 * P(wi, wi-1) + lambda2 * P(wi) on lambda0 + lambda1 + lambda2 = 1

El pes de cada lambda depén de quina es la millor probabilitat donat el corpus. La P basada en enegrames més llargs és més precisa pero si el corpus es mes petit, anira malament...

- Altra: Discount

Pabs(X =x) = (C(X=x)-delta)/N if C(w1...wn) > 0
            ((B-No)*delta/No)/N otherwise

- Altra: Backing-off

Si trobem el enegrama fem servir la seva prob. Si no, la del n-1 grama. Si no, la del n-2 grama...
S'han d'aproximar una certa alfa i una certa belta per aasegurar que el sumatori de les probabilitats siguin 1.
Es poden donar valors aporximats i ja està, normalment es farien amb good tuning.

- Probar con diferente numero de enegramas de cara al análisis



**Càrrega de dades**

In [ ]:
import spacy

from pathlib import Path


In [5]:
path = Path(r"C:\Users\Aya\UPC\PLH\Projecte 1\langId\eng_trn.txt")
text = path.read_text(encoding="utf-8")


**Preprocés**

***Eliminar els dígits***

In [9]:
import re

text_clean = re.sub(r"\d+", "", text)  # eliminar números

text = text_clean

***Convertir el text a minúscula***

In [10]:
text = text.lower()  # convertir a minúsculas

***Substituir espais en blanc per un sol***

In [11]:
text = re.sub(r"\s+", " ", text).strip()  # eliminar espacios extra

**Eliminar caràcters extranys**

In [ ]:
weird_chars = ["—", "…", ':', "“", "”", "‘", "’", "•", "–", "—", "―", "«", "»", '%', '&', '$', '@', '(', ')', '[', ']', '{', '}', '|', '\\', '/', '<', '>', '^', '~', '`']
for char in weird_chars:
    text = text.replace(char, " ")

***Concatenar totes les frases amb un espai doble al mig***

In [13]:
sentences = [line.strip() for line in text.splitlines() if line.strip()]
text_joined = "  ".join(sentences)
text = text_joined
print(text[:500])  # vistazo rápido

. .  how the mighty have fallen. . .  major companies coming out with their latest quarterly numbers include dave   buster's, men's wearhouse, palo alto networks, box, barnes   noble, pep boys-manny, moe   jack, ollie's bargain outlet, davidstea, lululemon athletica and kroger. january , magazine why are there so many magna cartas? : now watching up next word association with brad woodhouse the president of the democratic american bridge pac on the gop presidential field. : autoplay autoplay cop


DEJAR DE MOMENTO

In [ ]:
import re, math
from pathlib import Path
from collections import Counter

# ─── Función de preprocesado completa ───────────────────────────────────

weird_chars = [
    "—", "…", ":", "\u201c", "\u201d", "\u2018", "\u2019", "•", "–",
    "―", "«", "»", "%", "&", "$", "@", "(", ")", "[", "]", "{", "}",
    "|", "\\", "/", "<", ">", "^", "~", "`", "#", "=", "+", "*", "_",
]

def preprocess(raw_text):
    """Aplica todos los pasos de preprocesado al texto crudo."""
    processed = []
    for line in raw_text.splitlines():
        line = re.sub(r"\d+", "", line)          # eliminar dígitos
        line = line.lower()                       # minúsculas
        for ch in weird_chars:
            line = line.replace(ch, " ")
        line = re.sub(r"\s+", " ", line).strip()  # colapsar espacios
        if line:
            processed.append(line)
    # Concatenar frases con doble espacio (marca de frontera)
    return "  ".join(processed)

# ─── Cargar y preprocesar todos los idiomas ─────────────────────────────

languages = ["deu", "eng", "fra", "ita", "nld", "spa"]
base_path = Path("langId")

train_texts = {}
test_texts  = {}

for lang in languages:
    train_texts[lang] = preprocess((base_path / f"{lang}_trn.txt").read_text(encoding="utf-8"))
    test_texts[lang]  = preprocess((base_path / f"{lang}_tst.txt").read_text(encoding="utf-8"))

print("Idiomas cargados:")
for lang in languages:
    print(f"  {lang}  →  train {len(train_texts[lang]):>8,} chars | test {len(test_texts[lang]):>7,} chars")

**Divisió en trigrames**

**Model d'enegrames**

In [ ]:
enegrames = []

In [ ]:
vocabulari = set(text)
B = len(vocabulari)

lamda = 0.5

def probabilitat_linstone(enegrama, lamda):
    count_enegrama = enegrames.count(enegrama)
    minigrama = enegrama[:-1]
    count_minigrama = enegrames.count(minigrama)

    def log_prob(self, text):
        """Log-probabilidad del texto con backing-off."""
        n = 3
        padded = " " * (n - 1) + text + " " * (n - 1)
        total = 0.0
        for i in range(len(padded) - n + 1):
            trigram  = padded[i:i+n]
            ctx_bi   = trigram[:2]          # contexto bigrama  (c_{i-2} c_{i-1})
            bigram   = trigram[1:]          # bigrama           (c_{i-1} c_i)
            ctx_uni  = trigram[1]           # contexto unigrama (c_{i-1})
            unigram  = trigram[2]           # unigrama          (c_i)

            ct_tri    = self.tri.get(trigram, 0)
            ct_ctx_bi = self.bi.get(ctx_bi, 0)

            if ct_tri > 0 and ct_ctx_bi > 0:
                # ── Nivel 1: trigrama ────────
                prob = ct_tri / ct_ctx_bi
            else:
                ct_bi      = self.bi.get(bigram, 0)
                ct_ctx_uni = self.uni.get(ctx_uni, 0)
                if ct_bi > 0 and ct_ctx_uni > 0:
                    # ── Nivel 2: backoff a bigrama ──
                    prob = self.alpha * (ct_bi / ct_ctx_uni)
                else:
                    # ── Nivel 3: backoff a unigrama (con add-1) ──
                    ct_uni = self.uni.get(unigram, 0)
                    prob = self.beta * ((ct_uni + 1) / (self.N_uni + self.B))

            total += math.log(prob) if prob > 0 else math.log(1e-10)
        return total

print("Clase BackoffModel definida ✓")

**Experimentació: selecció del millor λ (Lidstone)**

In [ ]:
# ─── Experimentación: Lidstone con diferentes λ ─────────────────────────
# Evaluamos accuracy de identificación de idioma sobre las frases de test.

lambdas_to_test = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
N_SENTS = 100  # frases de test por idioma (para reducir tiempo)

lidstone_results = {}

for lam in lambdas_to_test:
    # Construir un modelo Lidstone por idioma
    models = {lang: LidstoneModel(lang_ngrams[lang], B, lam=lam) for lang in languages}

    correct, total = 0, 0
    for true_lang in languages:
        sentences = [s.strip() for s in test_texts[true_lang].split("  ") if len(s.strip()) >= 10]
        for sent in sentences[:N_SENTS]:
            predicted = max(languages, key=lambda l: models[l].log_prob(sent))
            correct += (predicted == true_lang)
            total   += 1

    acc = correct / total
    lidstone_results[lam] = acc
    print(f"  λ = {lam:<6}  →  Accuracy = {acc:.4f}  ({correct}/{total})")

best_lam = max(lidstone_results, key=lidstone_results.get)
print(f"\n  ★ Mejor λ = {best_lam}  con accuracy = {lidstone_results[best_lam]:.4f}")

**Experimentació: selecció de α i β (Backing-off)**

In [ ]:
# ─── Experimentación: Backing-off con diferentes α, β ────────────────────

alpha_beta_pairs = [
    (0.2, 0.05), (0.3, 0.1), (0.4, 0.1),
    (0.4, 0.4),  (0.5, 0.2), (0.6, 0.3),
]

backoff_results = {}

for alpha, beta in alpha_beta_pairs:
    models = {lang: BackoffModel(lang_ngrams[lang], B, alpha=alpha, beta=beta) for lang in languages}

    correct, total = 0, 0
    for true_lang in languages:
        sentences = [s.strip() for s in test_texts[true_lang].split("  ") if len(s.strip()) >= 10]
        for sent in sentences[:N_SENTS]:
            predicted = max(languages, key=lambda l: models[l].log_prob(sent))
            correct += (predicted == true_lang)
            total   += 1

    acc = correct / total
    backoff_results[(alpha, beta)] = acc
    print(f"  α={alpha}, β={beta}  →  Accuracy = {acc:.4f}  ({correct}/{total})")

best_ab = max(backoff_results, key=backoff_results.get)
print(f"\n  ★ Mejor (α, β) = {best_ab}  con accuracy = {backoff_results[best_ab]:.4f}")

### Justificació dels valors de N i B

**N = 3 (trigrames de caràcters):**

- Els trigrames capturen patrons locals característics de cada idioma (p.ex. *"sch"* en alemany, *"the"* en anglès, *"les"* en francès, *"ció"* en espanyol).
- Els bigrames (N=2) tenen poca capacitat discriminativa: moltes parelles de caràcters són compartides entre idiomes.
- Els 4-grames (N=4) serien més específics però pateixen de *data sparsity*: molts 4-grames apareixen 0 o 1 vegades, especialment en corpus petits, la qual cosa augmenta la dependència del suavitzat i pot empitjorar la generalització.
- N=3 és l'estàndard en la identificació automàtica d'idiomes basada en caràcters (*Cavnar & Trenkle, 1994*) perquè ofereix el millor equilibri entre **especificitat** (prou context per diferenciar idiomes) i **cobertura** (suficients observacions per estimar probabilitats fiables).

**B = |V| (mida del vocabulari de caràcters):**

- B és el nombre de caràcters únics presents en el corpus conjunt de tots els idiomes.
- S'utilitza un vocabulari **compartit** perquè volem que tots els models siguin comparables: així, un caràcter no vist en un idioma no rep probabilitat 0 gràcies al suavitzat.
- En la fórmula de Lidstone: $P(c_i \mid c_{i-2}c_{i-1}) = \frac{C(\text{tri}) + \lambda}{C(\text{ctx}) + B \cdot \lambda}$, B controla quanta massa de probabilitat es redistribueix als n-grames no observats.
- Un B gran (molts caràcters: accents, diacrítics) dilueix més la probabilitat, però reflecteix millor la diversitat real dels idiomes europeus.